<a href="https://colab.research.google.com/github/GustavoGerenaTamara/Trabajos_programacion/blob/main/Clasificadores_Gustavo_Gerena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UNIVERSIDAD DISTRITAL FRANCISCO JOSÉ DE CALDAS
## TÓPICOS AVANZADOS EN AUTOMÁTICA 2022-3
### MAESTRÍA EN INGENIERÍA
### Gustavo Gerena

---
### MÉTODOS DE CLASIFICACIÓN

Para realizar la evaluación de los métodos de clasificación vistos, se realiza el siguiente paso a paso:

1. Seleccionar un conjunto de datos, en este caso es una base de datos que se extrajo de: https://archive.ics.uci.edu/ml/datasets/arrhythmia;
esta contiene información sobre las condiciones que podrían servir para predecir si se puede presentar o no una arritmia.
 
Selección de columnas relevantes
1 Age: Age in years , linear
3 Height: Height in centimeters , linear
4 Weight: Weight in kilograms , linear
5 QRS duration: Average of QRS duration in msec., linear
10 QRS 
Selección de columna de salida
10 QRS
En los electrocardiogramas los valores normales hallados son la frecuencia cardiaca con un valor mínimo de 60 y máximo de 80, la duración de QRS de 68 a 100, el intervalo QT de 314 a 430, intervalo PR, 124 a 189, onda P 36 a 120, intervalo RR 615 a 1000 y su eje de QRS de -28 a 94.


Se dividen los datos para entrenamiento y pruebas, para ello se usarán 70%/30%,  para evitar el overfittitng 
Fuente: https://empresas.blogthinkbig.com/datos-entrenamiento-vs-datos-de-test/#:~:text=Normalmente%20el%20conjunto%20de%20datos,el%20sobreajuste%20u%20%E2%80%9Coverfitting%E2%80%9D.
Balancear BD 

Cargamos el dataset disponible en https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data ; esta base de datos contiene 279 atributos, 206 de los cuales son de valor lineal y el resto son nominales.

El objetivo es distinguir entre la presencia y ausencia de arritmia cardíaca y clasificarla en uno de los 16 grupos. La clase 01 se refiere a ECG 'normal'. Las clases 02 a 15 se refieren a diferentes clases de arritmia y la clase 16 se refiere al resto de las no clasificadas

In [ ]:
#Cargamos los datos de una carpeta local
f_in = open("arrhythmia.data",'r')
arrhythmia = f_in.readlines()
f_in.close()


In [ ]:
import pandas as pd
import numpy as np

alturas_cm=[]
pesos_kg=[]
Frecuencia_cardiaca=[] #Número de latidos cardíacos por minuto
Num_de_deflex=[]
T = [] #Duración media de la onda T en mseg
P = [] #Duración media de la onda P en mseg
Onda_Q=[] #Ancho promedio, en mseg onda Q
Onda_R=[] #Ancho promedio, en mseg onda R
Onda_S=[] #Ancho promedio, en mseg onda S
P_R = [] #duración promedio entre el inicio de las ondas P y R en mseg
Q_T = [] #duración promedio entre el inicio de Q y el final de las ondas T en mseg
grupos=[]
for renglon in arrhythmia:
  datos_ren_str = renglon.split(',')
  try:
      altura_cm = int(datos_ren_str[2])
      if altura_cm > 200:
        continue
      peso_kg = int(datos_ren_str[3])
      p_r = int(datos_ren_str[5])
      q_t = int(datos_ren_str[6])
      t = int(datos_ren_str[7])
      p = int(datos_ren_str[8])
      frecuencia_cardiaca = int(datos_ren_str[14])
      Onda_q = int(datos_ren_str[15])
      Onda_r = int(datos_ren_str[16])
      Onda_s = int(datos_ren_str[17])
      num_de_deflex = int(datos_ren_str[20]) 
      grupo = int(datos_ren_str[-1])
  except:
     continue
  alturas_cm.append(altura_cm)
  pesos_kg.append(peso_kg)
  Frecuencia_cardiaca.append(frecuencia_cardiaca)
  Num_de_deflex.append(num_de_deflex)
  T.append(t)
  P.append(p)
  Onda_Q.append(Onda_q)
  Onda_R.append(Onda_r)
  Onda_S.append(Onda_s)
  P_R.append(p_r)
  Q_T.append(q_t)
  grupos.append(grupo)

minima,maxima=min(pesos_kg),max(pesos_kg)
peso_car=[int((i-minima)/10) for i in pesos_kg  ]

minima,maxima=min(alturas_cm),max(alturas_cm)
altura_car=[int((i-minima)/10) for i in alturas_cm  ]

minima,maxima=min(Frecuencia_cardiaca),max(Frecuencia_cardiaca)
Frecuencia_cardiaca_car=[int((i-minima)/10) for i in Frecuencia_cardiaca  ]

minima,maxima=min(Num_de_deflex),max(Num_de_deflex)
Num_de_deflex_car=[int((i-minima)/10) for i in Num_de_deflex  ]

minima,maxima=min(T),max(T)
T_car=[int((i-minima)/10) for i in T  ]

minima,maxima=min(P),max(P)
P_car=[int((i-minima)/10) for i in P  ]

minima,maxima=min(Onda_Q),max(Onda_Q)
Onda_Q_car=[int((i-minima)/10) for i in Onda_Q  ]

minima,maxima=min(Onda_R),max(Onda_R)
Onda_R_car=[int((i-minima)/10) for i in Onda_R  ]

minima,maxima=min(Onda_S),max(Onda_S)
Onda_S_car=[int((i-minima)/10) for i in Onda_S  ]

minima,maxima=min(P_R),max(P_R)
P_R_car=[int((i-minima)/10) for i in P_R ]

minima,maxima=min(Q_T),max(Q_T)
Q_T_car=[int((i-minima)/10) for i in Q_T ]

minima,maxima=min(grupos),max(grupos)
grupos_car=[i-1 for i in grupos ]



#data = pd.DataFrame(list(zip(altura,peso,Frecuencia_cardiaca,Num_de_deflex,T_car,P,Onda_Q,Onda_R,Onda_S,P_R,Q_T,grupos)),
 #                           columns =['alturas_cm', 'pesos_kg','Frecuencia_cardiaca','Num_de_deflex','T','P','Onda_Q','Onda_R','Onda_S','P_R','Q_T','Salida']) # No escalados
data = pd.DataFrame(list(zip(altura_car,peso_car,Frecuencia_cardiaca_car,Num_de_deflex_car,T_car,P_car,Onda_Q_car,Onda_R_car,Onda_S_car,P_R_car,Q_T_car,grupos_car)),
                            columns =['alturas_cm', 'pesos_kg','Frecuencia_cardiaca','Num_de_deflex','T','P','Onda_Q','Onda_R','Onda_S','P_R','Q_T','Salida'])
data.head()

##Pre-procesamiento de datos
Ya que utilizaremos un metodo de clasificación basada en regresión logistica, es necesario ajustar la salida a 0 o 1, donde:
<table>
  <tr>
    <th>Salida</th>
    <th>Salida ajustada</th>
  </tr>
  <tr>
    <td>1</td>
    <td>Paciente sin arritmia (0)</td>   
  </tr>
  <tr>
    <td>2-16</td>
    <td>Paciente con arritmia (1)</td>   
  </tr>
   
</table>
Lo anterior indica que si la salida es 0 el diagnostico es negativo, de lo contrario el diagnostico es positivo.

In [ ]:
# Realizamos el preprocesamiento de la salida
Pre_out = data['Salida'].copy() # Creamos una copia de la columna de salida.
for index, value in enumerate(grupos):
    if value == 1:
      Pre_out[index] = 0
    else:
      Pre_out[index] = 1
Pre_out

0      1
1      1
2      1
3      0
4      1
      ..
444    0
445    1
446    1
447    0
448    0
Name: Salida, Length: 449, dtype: int64

De esta forma aseguramos que la salida (y) sea un 1 o 0, de tal forma que podamos usar el metodo de regresión logistica como clasificador
Selección datos de entrada y salida del dataset creado previament


In [ ]:
X = data.iloc[:,:11] # Seleccionamos las primeras 11 columnas del DataFrame correspondiente a las entradas
y = Pre_out # Seleccionamos la Salida previamente modificada
m,n = X.shape
m,n

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
np.random.seed(7)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPClassifier
plt.rcParams['image.cmap'] = "bwr"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')
# División de los datos en train y test
# ==============================================================================
X_train, X_test, y_train, y_test = train_test_split(
                                       X,
                                       y,
                                        train_size   = 0.5,
                                        random_state = 1234,
                                        shuffle      = True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

nicialmente realizaremos el ejercicio de clasificación con los datos de entrada sin ningun tipo de modificación.
#Vecino mas cercano

In [ ]:
from scipy.linalg.decomp_svd import zeros
k_min, err_min = np.inf,np.inf
i = 200
ks, errores = np.zeros(i-1),np.zeros(i-1) # Declaramos los arreglos en donde se van a guardar los valores de k y el error
for k in range(1,i,1):
  model = KNeighborsRegressor(n_neighbors=k)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  error = mean_squared_error(y_test, y_pred)
  ks[k-1] = k # Guardamos los valores de en un arreglo
  errores[k-1] = error # Guardamos los valores de error de cada k en un arreglo
  # Evaluamos el error minimo y guardamos el valor de k de dicho valor
  if error<err_min:
    err_min = error
    kf = k
    rmse_vecinosn = mean_squared_error(y_test, y_pred)
print('El minimo error encontrado es',rmse_vecinosn,', cuyo valor de k es',kf)

##Bayes

In [ ]:
from sklearn.naive_bayes import CategoricalNB
X_train, X_test, y_train, y_test = train_test_split(
                                       X,
                                       y,
                                        train_size   = 0.5,
                                        random_state = 1234,
                                        shuffle      = True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


clf = CategoricalNB()
clf.fit(X_train, y_train)
rmse_Bayen = mean_squared_error(y_test, y_pred)
presBn = accuracy_score(y_train, clf.predict(X_train))
print('El error en la predicción corresponde a:',rmse_Bayen,',con una precisión de:',presBn)


#Regresión Logistica

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                       X,
                                       y,
                                        train_size   = 0.5,
                                        random_state = 1234,
                                        shuffle      = True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

modelo = LogisticRegression()
modelo.fit(X = X_train, y = y_train)
presRLn = accuracy_score(y_train, modelo.predict(X_train))
rmse_RLn = mean_squared_error(y_test, y_pred)
print('El error en la predicción corresponde a:',rmse_RLn,',con una precisión de:',presRLn)
